In [74]:
%pip install cryptography

Note: you may need to restart the kernel to use updated packages.


In [75]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.asymmetric import dh
import random

### Key exchange

In [82]:
P = 17;
G = 6;

c = random.randint(1, 1000000)
public_key_c = (G**c)%P 

with open('shared_file', 'wb') as f:
    f.write(public_key_c.to_bytes(2, "big"))
    f.close()


In [83]:
with open('shared_file', 'rb') as f:
    public_key_d = f.read()
    public_key_d = int.from_bytes(public_key_d, byteorder='big', signed=False)

sC = (public_key_d**c)%P

print(sC)

14


### Encryption

In [81]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.serialization import load_pem_public_key

In [89]:
controler_private_key = rsa.generate_private_key(public_exponent= 65537, key_size= 4096)
controler_public_key = controler_private_key.public_key()

pem = controler_public_key.public_bytes(
   encoding=serialization.Encoding.PEM,
   format=serialization.PublicFormat.SubjectPublicKeyInfo
)

with open('shared_file', 'wb') as f:
    f.write(pem)
    f.close()

In [88]:
with open("shared_file", "rb") as file:
    delentture_public_key = load_pem_public_key(file.read())

In [84]:
with open("data/columns_description.csv", 'r') as file:
  columns_description = file.read()

In [85]:
padding_obj = padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None)

encrypted_message = delentture_public_key.encrypt(columns_description.encode(), padding_obj)

ValueError: Encryption/decryption failed.